# Análisis del riesgo de incumplimiento de los prestatarios

Tu proyecto consiste en preparar un informe para la división de préstamos de un banco. Deberás averiguar si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

Tu informe se tendrá en cuenta al crear una **puntuación de crédito** para un cliente potencial. La **puntuación de crédito** se utiliza para evaluar la capacidad de un prestatario potencial para pagar su préstamo.

# Proposito

Hacer un análisis profundo de información para determinar las caracteristicas principales de los cliente potenciales a  incumplir sus obligaciones crediticias relacionadas con prestamos bancarios.

# Hipótesis

Carácteristicas como **número de hijos** y **estado civil** de diversos clientes tienen un gran impacto en el historial crediticio de las personas, llegando al punto de afirmar que las personas con **hijos** que se encuentran **casados** tienden a no cumplir sus obligaciones crediticias. 

## Abre el archivo de datos y mira la información general. 

Debes importar la librería **Pandas** para poder cargas los datos y trabajar con ellos haciendo uso de los diferentes métodos. Se hace un renombre de la librería con la finalidad de facilitar su invocación.

In [1]:
# Cargar todas las librerías
import pandas as pd


In [2]:
# Carga los datos
df = pd.read_csv('/datasets/credit_scoring_eng.csv')


## Ejercicio 1. Exploración de datos

**Descripción de los datos**
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo

Al ejecutar la `df.info()` nos damos cuenta que tenemos en total 21525 filas de datos, con tipos de datos `int64`, `float64` y `object`. De las columnas presentadas podemos concluir que hay un nombre de columna que no describe muy bien sus datos y es `dob_years` que cambiaría por `age`; para el caso de tipo de datos pasaría en la columna `days_employed` de `float64` a `int64` puesto que la columna describe los días que lleva empleado en la empresa más no la cantidad de tiempo en horas, minutos, etc., para el caso de la columna `debt` posiblemente podría usar un tipo de dato `bool` sin embargo se debe hacer más análisis al respecto. Aparentemente para el caso de las columnas `days_employed` y `total_income` se tienen valores `null`. En conclusión hasta este punto, es necesaria más exploración para determinar que es lo que se le aplicara a cada columna en primera estancia.

In [3]:
# Vamos a ver cuántas filas y columnas tiene nuestro conjunto de datos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
# vamos a mostrar las primeras filas N
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding



En la muestra podemos concluir algunas cosas:
- `days_employed`: Nos muestra en la mayoría de los casos (muestra de 10 filas) números negativos (requiere más exploración con la finalidad de determinar si solo es una mala aplicación de formula o son datos que no afectan nuestra investigación).
- `education`: Se puede observar que es información categórica, sin embargo, no tiene un formato específico, por lo tanto, debemos homologar dicha información de acuerdo a las categorías presentadas.
- `education_id`: Podemos observar que solo se cuenta con dos identificadores pues en la muestra de 10 filas, solo se observan 2 categorías aparentemente. Requiere más exploración.
- `family_status`: Aparentemente se tienen 2 categorías bien homologadas. Requiere confirmación de dicha observación.
- `family_status_id`: Aparentemente representa de forma correcta las categorías de la columna `family_status`. Requiere confirmación.
- `gender`: Apartenmente cuenta con 2 categorías bien homologadas. Requiere confirmación.
- `income_type`: Apartenmente cuenta con 2 categorías bien homologadas. Requiere confirmación.
- `debt`: Requiere validación para saber de que forma debe ser trabajada esta columna, es decir, se observa que es una variable con 2 categorías (tiene o no deuda), sin embargo se debe explorar a mayor profundidad con la intención de validar que efectivamente sea de esa forma.

In [5]:
# Obtener información sobre los datos
print(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64



Basados en la observación de `df.info()` se concluye que efectivamente tanto `days_employed` como `total_income` tiene valores `null` dando un total de 2174 en ambos casos.

In [6]:
# Veamos la tabla filtrada con valores ausentes de la primera columna donde faltan datos
df_days_employed = df[df['days_employed'].isna() == True].index
df_total_income = df[df['total_income'].isna() == True].index
print(list(df_days_employed) == list(df_total_income))

True



Generando `df` filtrados de acuerdo a los valores nulos y extrayendo el index que se encuentra en la columna `days_employed` y `total_income` para finalmente convertir en lista ese objeto y hacer una comparación podemos decir con toda seguridad que los valores ausentes en la columna `days_employed` corresponden a la misma posición que los valores ausentes encontrados en `total_income`. Lo que nos lleva a la primer afrimación:
- Aquellas filas que cuentan con valores nulos en la columna `days_employe` tiene valores nulos de igual forma en `total_income`

In [7]:
# Apliquemos múltiples condiciones para filtrar datos y veamos el número de filas en la tabla filtrada.
print(len(df_total_income)/len(df)*100)

10.099883855981417


**Conclusión intermedia**

Aparentemente, haciendo la comparación de la longitud del `df` original y el generado unicamente con los valores ausentes ya sea para las columnas `days_employed` o `total_income` el porcentaje que representan estos valores ausentes en el `df` original es del `10%` lo cual nos da a pensar que en un porcentaje que podría no repercutir en el análisis si es omitido debido a que es una muestra pequeña en comparación del resto, sin embargo, considero que podríamos profundizar con la intención de completar esos valores ausentes siempre y cuando se encuentre algún patrón o dependencia del resto de características.

In [8]:
# Vamos a investigar a los clientes que no tienen datos sobre la característica identificada y la columna con los valores ausentes

df[df['days_employed'].isna()].info()
print()
df[df['total_income'].isna()].info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2174 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------ 

In [9]:
# Comprobación de la distribución
# df[(df['days_employed'].isna()) | (df['total_income'].isna())].value_counts(subset=['days_employed','total_income'])

df.pivot_table(values='days_employed',index='family_status',columns='children')

children,-1,0,1,2,3,4,5,20
family_status,,,,,,,,
civil partnership,-3047.998983,76434.389740,16944.727119,-867.989149,6283.150953,-1199.174225,-387.317579,32252.608579
divorced,84420.369405,88934.980469,22334.152742,2936.252080,-1864.997352,-344.926495,NaN,-1044.201789
married,25314.034785,90495.966789,16884.973077,806.464841,3881.244354,14229.562654,-1581.638747,29219.100934
unmarried,175766.602281,49820.950087,19757.795012,9384.364154,-1659.035198,-716.443813,NaN,-935.885997
widow / widower,177902.189095,218083.622561,102762.608681,39206.729347,60409.854811,-4584.852621,NaN,235010.364067


In [10]:
# Agrupando por la columna 'children' y contabilizando cada categoría
df[df['days_employed'].isna()].groupby('children')['education'].count()
df[df['days_employed'].isna()].groupby('education_id')['children'].count()
df[df['days_employed'].isna()].sort_values(by='dob_years')['dob_years'].unique()
df[df['days_employed'].isna()].groupby('income_type')['children'].count()

income_type
business          508
civil servant     147
employee         1105
entrepreneur        1
retiree           413
Name: children, dtype: int64

In [11]:
# Agrupando por la columna 'education_id' y contabilizando cada categoría
df[df['days_employed'].isna()].groupby('education_id')['children'].count()


education_id
0     544
1    1540
2      69
3      21
Name: children, dtype: int64

In [12]:
# Agrupando por la columna 'dob_years' extrayendo los valores únicos
df[df['days_employed'].isna()].sort_values(by='dob_years')['dob_years'].unique()


array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73])

In [13]:
# Agrupando por la columna 'income_type' y contabilizando cada categoría
df[df['days_employed'].isna()].groupby('income_type')['children'].count()

income_type
business          508
civil servant     147
employee         1105
entrepreneur        1
retiree           413
Name: children, dtype: int64


**Posibles razones por las que hay valores ausentes en los datos**

Examinando un poco más el `df` unicamente para aquellas filas que cuentan con valores ausentes tanto en `days_employed` como en `total_income` en conjunto con `education_id`, se observa que estos valores son más presentes en aquellos con `Secondary Education` como característica dando un total de 1540, seguido por `Bachelor's Degree` con 544, despues por `Some College` con un total de 69 y por último `Primary Education` con 21.
Se podría decir lo siguiente:
- Se encuentran desempleados
- Sus negocios han quebrado
- Estan retirados

In [14]:
# Comprobando la distribución en el conjunto de datos entero

print('Conjunto de datos completo:')
print(df.groupby('education_id')['education'].count() / len(df) * 100)
print()
print('Conjunto de datos filtrado:')
print(df[df['days_employed'].isna()].groupby('education_id')['education'].count() / len(df[df['days_employed'].isna()]) * 100)

Conjunto de datos completo:
education_id
0    24.436702
1    70.768873
2     3.456446
3     1.310105
4     0.027875
Name: education, dtype: float64

Conjunto de datos filtrado:
education_id
0    25.022999
1    70.837167
2     3.173873
3     0.965961
Name: education, dtype: float64


**Conclusión intermedia**

Como podemos observar al hacer una comparación del `df` original y el filtrado, vemos distribuciones similares, lo cual no nos permite visualizar algún parámetro que este relacionado con la ausencia de estos datos.

In [15]:
# Comprueba otras razones y patrones que podrían llevar a valores ausentes

df[(df['days_employed'].isna()) & (df['dob_years'] > 21)].groupby('education')['children'].count()

education
BACHELOR'S DEGREE        23
Bachelor's Degree        23
PRIMARY EDUCATION         1
Primary Education         1
SECONDARY EDUCATION      67
SOME COLLEGE              6
Secondary Education      65
Some College              7
bachelor's degree       490
primary education        19
secondary education    1383
some college             55
Name: children, dtype: int64

**Conclusión intermedia**

Al generar un `df` nuevo en el cuál se considera la edad y que sea mayor a 21 años, esto con la finalidad de asegurar que se trate de personas que podrían trabajar sin restricciones de edad, nos damos cuenta que en todos los niveles educativos se tienen registros con valores nulos en `days_employed`. Por esta razón aun no es claro a que se deban estos valores ausentes sin embargo considero que no son accidentes, podría ser una falla muy clara a la hora de hacer el registro de datos, importarlos, etc.

In [16]:
# Comprobación de otros patrones: explica cuáles

df[df['days_employed'].isna()].sort_values(by='dob_years')['dob_years'].unique()

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73])

**Conclusiones**

Hasta este punto no se ha logrado encontrar algún patrón específico por el cual sea posible identificar que valores son los ausentes en los registros. Se identifico que aquellos registros en los que se encuentra ausente el valor correspondiente a la columna de `days_employed` corresponden en su totalidad a valores ausentes en la columna `total_income`; esto nos llevo a la pregunta en la que se podrían relacionar estas columnas de acuerdo al grado de estudio, teniendo como presima que posiblemente los datos faltantes se deban a registros de estudiantes unicamente, sin embargo, al filtrar los datos se encontro que dentro de estos valores ausentes se encontraban todas las categorías correspondientes a la columna `education`. Esto aun no contestaba del todo la pregunta, por ende, se relacionaron estos registros con la columna `dob_years` que nos muestra la edad del trabajador y practicamente el 100% de los registros se encuentra de un rango de edad en el que en condiciones esperadas es posible laborar.
Por último al relacionar estos valores con la columna `type_income` con la esperanza de encontrarse con categorías que se refirieran a personas retiradas o cosas similares, nos encontramos con la misma problematica en la que indiscriminadamente se tienen de todo tipo de categorías englobadas en dichos registros.

## Transformación de datos

In [17]:
# Veamos todos los valores en la columna de educación para verificar si será necesario corregir la ortografía y qué habrá que corregir exactamente
df['education'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [18]:
# Arregla los registros si es necesario
df.loc[(df['education'] == "BACHELOR'S DEGREE") | (df['education'] == "Bachelor's Degree"),['education']] = "bachelor's degree"
df.loc[(df['education'] == "SECONDARY EDUCATION") | (df['education'] == "Secondary Education"),['education']] = "secondary education"
df.loc[(df['education'] == "SOME COLLEGE") | (df['education'] == "Some College"),['education']] = "some college"
df.loc[(df['education'] == "PRIMARY EDUCATION") | (df['education'] == "Primary Education"),['education']] = "primary education"
df.loc[(df['education'] == "GRADUATE DEGREE") | (df['education'] == "Graduate Degree"),['education']] = "graduate degree"


In [19]:
# Comprobar todos los valores en la columna para asegurarnos de que los hayamos corregido

df['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

In [20]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


In [21]:
# Veamos la distribución de los valores en la columna `children`

print('Conjunto de datos completo:')
print(df.groupby('children')['education'].count() / len(df) * 100)
print()
print('Conjunto de datos filtrado:')
print(df[df['days_employed'].isna()].groupby('children')['education'].count() / len(df[df['days_employed'].isna()]) * 100)

Conjunto de datos completo:
children
-1      0.218351
 0     65.732869
 1     22.383275
 2      9.547038
 3      1.533101
 4      0.190476
 5      0.041812
 20     0.353078
Name: education, dtype: float64

Conjunto de datos filtrado:
children
-1      0.137994
 0     66.191352
 1     21.849126
 2      9.383625
 3      1.655934
 4      0.321987
 5      0.045998
 20     0.413983
Name: education, dtype: float64


In [22]:
df.pivot_table(values='education_id',columns=['children'],index=['education'],aggfunc='count')
df.groupby(['education'])['education_id'].count()/len(df) * 100
df.groupby(['children'])['education_id'].count()/len(df) * 100

children
-1      0.218351
 0     65.732869
 1     22.383275
 2      9.547038
 3      1.533101
 4      0.190476
 5      0.041812
 20     0.353078
Name: education_id, dtype: float64


Para el caso de la columna `children` y tratandose del set de datos completo, se visualiza algo inusual, la columna describe la cantidad de hijos que tiene el cliente, por ende se entiende que el rango va de 0 en adelante, sin embargo hay registros que tienen como dato un `-1` cosa que no es correcta; haciendo una evaluación del porcentaje de registros que se tiene tanto en el `df` completo como el filtrado (aquel que tiene los valores ausententes anteriormente mencionados) se encuentra que es de 0.21% y 0.13% respectivamente, esto equivale a un máximo de 47 registros afectados por este dato.

Por otro lado observamos también que se tiene un total de 0.35% y 0.41% de registros con un total de 20 hijos para el `df` original y el filtrado respectivamente. Esto no lleva a pensar en algo muy complicado de ser verdad, sin embargo, podriamos conjuntar estos datos.

Utilizando los métodos `group_by` y `pivot_table` nos encontramos con dos hechos que podrían ayudar a definir que sucedera con los valores anómalos en la tabla `children`, es decir, valores para `-1`, `5` y `20`, por un lado, podemos observar que el porcetaje correspondiente a estos registros es mínimo, `0.21%`, `0.04%` y `0.035%` respectivamente

Considero que al ser una muestra diminuta en comparación con el resto, es posible cambiar todos aquellos registros en los que se encuentre el -1 por 0 en la columna `children` y agrupar en una sola categoría aquellos que tiene 5 o más hijos.

In [23]:
# [arregla los datos según tu decisión]
df['children'].loc[df['children'] == -1] = 0

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [24]:
df['children'].loc[df['children'] == 20] = 5
df['children'].loc[df['children'] == 5] = '5 o más'

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [25]:
# Comprobar la columna `children` de nuevo para asegurarnos de que todo está arreglado

print('Conjunto de datos completo:')
print(df.groupby('children')['education'].count() / len(df) * 100)
print()
print('Conjunto de datos filtrado:')
print(df[df['days_employed'].isna()].groupby('children')['education'].count() / len(df[df['days_employed'].isna()]) * 100)

Conjunto de datos completo:
children
0          65.951220
1          22.383275
2           9.547038
3           1.533101
4           0.190476
5 o más     0.394890
Name: education, dtype: float64

Conjunto de datos filtrado:
children
0          66.329347
1          21.849126
2           9.383625
3           1.655934
4           0.321987
5 o más     0.459982
Name: education, dtype: float64


In [26]:
# Encuentra datos problemáticos en `days_employed`, si existen, y calcula el porcentaje

print(list(df.sort_values(by=['days_employed'])['days_employed'].unique()))

[-18388.949900568383, -17615.563265627912, -16593.472817263817, -16264.699500887124, -16119.68773669392, -15835.725774811905, -15785.678893355003, -15773.0613349239, -15618.063786344595, -15410.040779095478, -15267.541182734109, -15193.032201443106, -15079.216068566157, -14920.049805042863, -14747.887934053397, -14677.9022934147, -14652.093785926449, -14583.15250401094, -14492.543986842333, -14447.661078047651, -14420.005180584703, -14381.545843648422, -14240.932399924128, -14103.863413698578, -14051.20262056069, -13976.806877265004, -13967.98538897768, -13923.375086079388, -13913.41587629005, -13894.357289777596, -13840.953867998049, -13809.045625637127, -13724.223883918732, -13710.428182355554, -13702.636708175783, -13664.197881775632, -13623.026318165186, -13563.100420050008, -13489.372942477548, -13468.882696747742, -13459.725039589568, -13424.333742690078, -13372.43469299878, -13359.000636018189, -13343.010652531377, -13264.142142731776, -13210.485011656738, -13151.226127616395, -



De entrada lo que se puede encontrar a simple vista es que hay días negativos, lo cuál no correspondería a información confiable, podríamos asumir de momento que tal vez simplemente a los valores se les asigno el signo negativo, sin embargo nos encontramos con que también hay valores positivos. El rango de valores va desde `-18388.949900568383` hasta `401755.40047533`, incluyendo los valores `nan`.

Por otro lado nos encontramos también con datos completamente anómalos, es decir, del rango superior anteriormente mencionado, al contabilizar los años contenidos en esta cantidad de días nos da como resultado `1,100.69973 años` lo cuál es un dato completamente fuera de rango.

En conclusión hasta este punto:
- Contamos con datos númericos flotantes tanto negativos como positivos, cuando deberían ser enteros positivos
- Hay valores que no corresponen a la cantidad de días laborando con la edad que tiene el cliente
- Además continuamos con el problema de los resgistros nulos.

In [27]:
# Aborda los valores problemáticos, si existen.

# Extrayendo media y mediana de los datos.
df_days_employed_mean = df['days_employed'].mean()
df_days_employed_median = df['days_employed'].median()

# Contabilizando la cantidad de datos positivos y negativos
negative_days_employed = df[df['days_employed'] < 0]['children'].count()
positive_days_employed = df[df['days_employed'] >= 0]['children'].count()

# Extrayendo la cantidad de días que no tienen sentido asumiendo que todos son positivos
stranger_values = df.copy()
stranger_values['days_employed'] = stranger_values['days_employed'].abs()
stranger_values = stranger_values[stranger_values['dob_years'] > 0]
stranger_values = stranger_values[stranger_values['days_employed']/365 <= stranger_values['dob_years']]

print(f'La media para "days_employed" es {df_days_employed_mean} y la mediana es {df_days_employed_median}.')
print(f'La cantidad de datos negativos en "days_employed" es {negative_days_employed} y positivos es {positive_days_employed}.')
print(f'Si asumimos que "days_employed" es positivo en todo momento, se tiene un total de {len(stranger_values)} registros válidos, es decir, {len(stranger_values)/len(df):0%} podría ser trabajado bajo esta condición.')


La media para "days_employed" es 63046.49766147338 y la mediana es -1203.369528770489.
La cantidad de datos negativos en "days_employed" es 15906 y positivos es 3445.
Si asumimos que "days_employed" es positivo en todo momento, se tiene un total de 15832 registros válidos, es decir, 73.551684% podría ser trabajado bajo esta condición.


In [28]:
# Generando una columna que almacene el signo de los registros de `days_employed` para futuras referencias

df['days_employed_signed'] = pd.Series(['+' if i >=0 else '-' for i in df['days_employed']])

# Generando una columna que almacene la cantidad de años trabajados basado en `days_employed`
df['employed_years'] = df['days_employed'].abs()
for i in range(len(df['employed_years'])):
    try:
        df['employed_years'][i] = df['employed_years'][i]/365
    except:
        df['employed_years'][i] = float('nan')

/tmp/ipykernel_41/2139590629.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['employed_years'][i] = df['employed_years'][i]/365


In [29]:
# Comprueba el resultado - asegúrate de que esté arreglado

df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_signed,employed_years
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,-,23.116912
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,-,11.026860
2,0,-5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,-,15.406637
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,-,11.300677
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,+,932.235814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,-,12.409087
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,+,942.294258
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,-,5.789991
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,-,8.527347


In [30]:
# Analizando los casos en los que la cantidad de años trabajados es incongruente
df[df['employed_years'] > 74].sort_values('employed_years').groupby('employed_years')['children'].count()

employed_years
900.626632     1
900.643627     1
900.743401     1
900.810210     1
900.896837     1
              ..
1100.448904    1
1100.477991    1
1100.479708    1
1100.591265    1
1100.699727    1
Name: children, Length: 3445, dtype: int64

Haciendo uso de información extraida de la red (`Esperanza de vida: Wikipedia`) para comenzar analisar aquellos valores en los que la cantidad de años es incongruente, consultamos nuestros datos enfatizando en un filtro en el que detectemos aquellos registros que tienen una cantidad de años superior a la esperanza de vida promedio mundial (`74`) años y nos encontramos que el primer valor atipico relacionado con esta información parte de `900.626632` en adelante y un total de `3445` registros afectados.

In [31]:
# Encontrando valores contrarios que esten muy cerca del valor promedio de esperanza de vida (de 59 a 74 años)
df[(df['employed_years'] > 59) & (df['employed_years'] < 74)].sort_values('employed_years').groupby('employed_years')['children'].count()


Series([], Name: children, dtype: int64)

Con esta lína de código, nos aseguramos que los valores atípicos son unicamente los valores presentados anteriormente puesto que registros que tengan cantidad de años muy cercano al promedio de esperanza de vida, es un filtrado nulo.

In [32]:
# Agrupando filtro con respecto a la columna "income_type" y contabilizando 
df[df['employed_years'] > 74].groupby('income_type')['children'].count()

income_type
retiree       3443
unemployed       2
Name: children, dtype: int64

Al agrupar el `df` filtrado, nos percatamos de algo interesante cuando usamos como parámetro la columna `income_type`. Las únicas categorías que tienen esta condición son `retiree` con `3443` registros afectados y `unemployed` con `2`.

Consideraremos lo siguiente para evaluar como corregir estos datos:
- Los días proporcionados fueron capturados erroneamente desde el punto decimal, es decir, debe recorrerse `n` lugares hacia la izquierda.
- La captura de registros pudo haber sido en una unidad distinta, es decir, en horas.
- Tenga alguna relación directa con la columna `dob_years` que debe analizarse.

In [33]:
# Evaluando el primer punto de corrección de datos (trabajando con "unemployed")
df_dyas_employed_unemployed = df[(df['employed_years'] > 74) & (df['income_type'] == 'unemployed')]['days_employed']*0.01/365
df_dyas_employed_unemployed.sort_values()


3133      9.247246
14798    10.830215
Name: days_employed, dtype: float64

In [34]:
# Evaluando el primer punto de corrección de datos (trabajando con "retiree")
df_dyas_employed_retiree = df[(df['employed_years'] > 74) & (df['income_type'] == 'retiree')]['days_employed']*0.01/365
df_dyas_employed_retiree.sort_values()


20444     9.006266
9328      9.006436
17782     9.007434
14783     9.008102
7229      9.008968
           ...    
7794     11.004489
2156     11.004780
7664     11.004797
10006    11.005913
6954     11.006997
Name: days_employed, Length: 3443, dtype: float64

In [35]:
# Evaluando resultados para filtrado de "unemployed" y "retiree"

for i in df_dyas_employed_retiree:
    if i > 74:
        print("Retiree: ",i,i.index)

for i in df_dyas_employed_unemployed:
    if i > 74:
        print("Unemployed: ",i,i.index)

In [36]:
# Evaluando resultados para filtrado de "retiree" excluyendo los casos atípicos
df[(df['employed_years'] < 74) & (df['income_type'] == 'retiree')]['employed_years'].sort_values()


Series([], Name: employed_years, dtype: float64)

Como se puede observar en la celda anterior, al recorrer el punto decimal 2 unidades hacia la izquierda, se generan valores que de forma lógica tienen sentido respecto a la cantidad de años trabajados, sin embargo, al ordenarlos nos damos cuenta que el rango de valores va desde los 9 años a los 11 años, lo cual para el caso de `unemployed` podría tener sentido.

Para el caso de `retiree` y debido a la poca cantidad de años trabajados en todos los casos podriamos concluir que:
- Alcanzarón alguna especie de libertad financiera a través de distintas inversiones.
- A causa de algún tipo de accidente laborar tubierón que jubilarlos a temprana edad.

In [37]:
# Aplicando el análisis de valores atípicos en la columna "days_employed" en relación con "dob_years"
df[(df['employed_years'] > 74) & (df['income_type'] == 'retiree')]['dob_years'].sort_values().unique()


array([ 0, 22, 26, 27, 28, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
       44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
       61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74])

Observando el resultado del filtrado de datos para aquellos registros con las condiciones anteriormente mencionadas, no podemos identificar algún patrón que relacione `days_employed` directamente con `dob_years`, por tal motivo, para poder trabajar estos registros anómalos, consideraremos como solución la opción de recorrer el punto decila 2 dígitos hacia la derecha en aquellos registros afectados.

In [38]:
# Aplicando análisis de valores atípicos en la columna "days_employed" cambiando la unidad con la que fue registrada
df_dyas_employed_retiree = (df[(df['employed_years'] > 74) & (df['income_type'] == 'retiree')]['days_employed']/24)/365
df_dyas_employed_retiree.sort_values()


20444    37.526110
9328     37.526818
17782    37.530975
14783    37.533759
7229     37.537368
           ...    
7794     45.852038
2156     45.853250
7664     45.853321
10006    45.857969
6954     45.862489
Name: days_employed, Length: 3443, dtype: float64

Como podemos observar en los resultados, pareciera ser que el factor que afecto estos registros fue la unidad, teniendo lógica respecto con la condición de la columna `type_income`. Por esta razón, se opto por reajustar los datos afectados bajo esta condición.

In [39]:
# Corrigiendo valores atípicos
df.loc[(df['employed_years'] > 74),'days_employed'] = (df['days_employed']/24)/365


Debido a este análisis, es necesario, reconstruir la columna `employed_years` puesta esta ha sido afectada, ademas que, la columna `days_employed_signed` deja de tener sentido, por lo tanto, se borrara.

In [40]:
# Corrigiendo valores de la columna 'employed_years' bajo el nuevo análisis 
df['employed_years'] = df['days_employed'].abs()
for i in range(len(df['employed_years'])):
    try:
        df['employed_years'][i] = df['employed_years'][i]/365
    except:
        df['employed_years'][i] = float('nan')
        
# Eliminando la columna 'days_employed_signed'
df.drop(['days_employed_signed'],axis=1)

/tmp/ipykernel_41/2712198015.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['employed_years'][i] = df['employed_years'][i]/365


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,employed_years
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,23.116912
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,11.026860
2,0,-5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,15.406637
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,11.300677
4,0,38.843159,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,0.106420
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,12.409087
21521,0,39.262261,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,0.107568
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,5.789991
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,8.527347


In [41]:
# Comprobando valores corregidos
df[df['employed_years'] > 74].groupby('income_type')['children'].count()


Series([], Name: children, dtype: int64)

Podemos observar que los valores han sido corregidos de forma exitosa, sin afectar el resto del código.

In [42]:
# Revisa `dob_years` en busca de valores sospechosos y cuenta el porcentaje

print(list(df.sort_values(by=['dob_years'])['dob_years'].unique()))

[0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]



El único valor realmente problemático que tienen los registros correspondiente a `dob_years` sería el `0` debido a que es imposible que una persona con 0 años de edad pueda trabajar. Por otro lado tal vez los adultos mayores a 65 años podrían considerarse valores problemáticos, sin embargo, esto puede ser debatible.

In [43]:
# Resuelve los problemas en la columna `dob_years`, si existen

df[df['dob_years'] == 0].head(15)
dob_years_mean = df.sort_values(by=['dob_years'])['dob_years'].mean()
dob_years_median = df.sort_values(by=['dob_years'])['dob_years'].median()
dob_years_max = df.groupby('dob_years')['children'].count().sort_values()

print(f'El promedio de edades es: {dob_years_mean}.')
print(f'La mediana de edades es: {dob_years_median}.')
print(f'La edad que más se repite es 35 con 617 casos, representando un {617/len(df):0%} de la muestra original.')

El promedio de edades es: 43.29337979094077.
La mediana de edades es: 42.0.
La edad que más se repite es 35 con 617 casos, representando un 2.866434% de la muestra original.


In [44]:
# Contando la cantidad de registros que tienen como valor "0" en la columna "dob_years"
df[df['dob_years'] == 0]['children'].count()

101

In [45]:
# Agrupando los datos anteriores de acuerdo a la columna "education" para los valores problematicos y en general
print(df[df['dob_years'] == 0].groupby('education')['children'].count())
print()
print(df.groupby('education')['children'].count())

education
bachelor's degree      35
secondary education    64
some college            2
Name: children, dtype: int64

education
bachelor's degree       5260
graduate degree            6
primary education        282
secondary education    15233
some college             744
Name: children, dtype: int64


In [46]:
# Mostrando las edades promedio de "dob_years" en relación con "education"
print(pd.pivot_table(df,index=['education'],values='dob_years',aggfunc='mean'))
print()
print(print(pd.pivot_table(df,index=['education'],values='dob_years',aggfunc='median')))


                     dob_years
education                     
bachelor's degree    40.668061
graduate degree      51.166667
primary education    47.691489
secondary education  44.536992
some college         34.661290

                     dob_years
education                     
bachelor's degree         39.0
graduate degree           51.5
primary education         46.5
secondary education       44.0
some college              31.0
None


In [47]:
# Extrayendo los valores promedio de "dob_years" para cada categoría en "education"

dob_years_mean_graduate = df[(df['education'] == "bachelor's degree") & (df['dob_years'] != 0)]['dob_years'].mean()
dob_years_mean_secondary = df[(df['education'] == 'secondary education') & (df['dob_years'] != 0)]['dob_years'].mean()
dob_years_mean_college = df[(df['education'] == 'some college') & (df['dob_years'] != 0)]['dob_years'].mean()

print(f'Promedio "Bachelors degree": {dob_years_mean_graduate}')
print(f'Promedio "Secondary Education": {dob_years_mean_secondary}')
print(f'Promedio "Some College": {dob_years_mean_college}')

Promedio "Bachelors degree": 40.94047846889952
Promedio "Secondary Education": 44.72489946601622
Promedio "Some College": 34.75471698113208


Debido a que `mean` y la `median` son similares podemos usar cualquier de ellos que en relación con la columna `education` podriamos determinar aproximadamente la edad que podría tener cada uno de los registros que tienen en la columna `dob_years` un valor de 0.

Las categorías que aparecen en nuestra tabla filtrada son:
- `bachelor's degree` con un promedio de edad de `51` años.
- `secondary education` con un promedio de edad de `44` años.
- `some college` con un promedio de edad de `34` años.

In [48]:
# Resolviendo el problema de registros "dob_years = 0"
df.loc[(df['dob_years'] == 0) & (df['education'] == "bachelor's degree"),'dob_years'] = dob_years_mean_graduate
df.loc[(df['dob_years'] == 0) & (df['education'] == 'secondary education'),'dob_years'] = dob_years_mean_secondary
df.loc[(df['dob_years'] == 0) & (df['education'] == 'some college'),'dob_years'] = dob_years_mean_college


In [49]:
# Comprueba el resultado - asegúrate de que esté arreglado
df[df['dob_years'] == 0]
df.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_signed,employed_years
0,1,-8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,-,23.116912
1,1,-4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,-,11.026860
2,0,-5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,-,15.406637
3,3,-4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,-,11.300677
4,0,38.843159,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,+,0.106420
5,0,-926.185831,27.0,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,-,2.537495
6,0,-2879.202052,43.0,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,-,7.888225
7,0,-152.779569,50.0,secondary education,1,married,0,M,employee,0,21731.829,education,-,0.418574
8,2,-6929.865299,35.0,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,-,18.985932
9,0,-2188.756445,41.0,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,-,5.996593


In [50]:
# Veamos los valores de la columna family_status

for i in df['family_status'].unique():
    print('{}: {}, Family Status Id: {}'.format(i,df[df['family_status'] == i]['children'].count(),df[df['family_status'] == i]['family_status_id'].unique()))

print()
print(df['family_status'].unique())
print(df['family_status_id'].unique())
    

married: 12380, Family Status Id: [0]
civil partnership: 4177, Family Status Id: [1]
widow / widower: 960, Family Status Id: [2]
divorced: 1195, Family Status Id: [3]
unmarried: 2813, Family Status Id: [4]

['married' 'civil partnership' 'widow / widower' 'divorced' 'unmarried']
[0 1 2 3 4]


En particular en la columna `family_status` no encuentro algún problema con los datos, estan perfectamente categorizados y corresponden con un único identificador en la columna `family_status_id`.

In [51]:
# Veamos los valores en la columna gender

df.groupby('gender')['children'].count()

gender
F      14236
M       7288
XNA        1
Name: children, dtype: int64

In [52]:
# Aborda los valores problemáticos, si existen
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_signed,employed_years
10701,0,-2358.600502,24.0,some college,2,civil partnership,1,XNA,business,0,32624.825,buy real estate,-,6.461919


En la columna `gender` nos encontramos con un registro que podría considerarse problema y es `XNA`, este puede pertenecer a cualquiera de los grupos `F` o `H` y debido a que no contamos con más información del mismo o algún patrón específico y que es solamente un registro, podemos añadilo al grupo más relevante que es `F`.

In [53]:
# Asignando "XNA" al grupo "F" de la columna "gender"
df.loc[(df['gender'] == 'XNA'),['gender']] = 'F'


In [54]:
# Comprueba el resultado - asegúrate de que esté arreglado
df[df['gender'] == 'XNA']
df.groupby('gender')['children'].count()


gender
F    14237
M     7288
Name: children, dtype: int64

In [55]:
# Veamos los valores en la columna income_type
df.groupby('income_type')['children'].count()


income_type
business                        5085
civil servant                   1459
employee                       11119
entrepreneur                       2
paternity / maternity leave        1
retiree                         3856
student                            1
unemployed                         2
Name: children, dtype: int64

Agrupando y contabilizando las categorías que se encuentran en la columna `type_income` nos encontramos con información que no es descriptiva y tal vez no tenga sentido.
Los puntos a abordar son:

- La categoría de `unemployed` esto debido a que, si una persona se encuentra desempleada, no es posible que tenga ingresos; por ende, es necesario aplicar otros filtros y validar información relacionada con `total_income` para saber que es lo que reporta este cliente.

En el caso de`student` que en este punto si bien no tiene ingresos a través de un trabajo puede tenerlos a través de becas y programas de ayuda gubernamentales, por ende, esta categoría la descartamos como valor problemático.

In [56]:
df[df['income_type'] == 'unemployed']['total_income']

3133      9593.119
14798    32435.602
Name: total_income, dtype: float64

El resultado del filtrado es incongruente puesto que en específico para aquellos clientes que reportan `income_type` como `unemployed` son 2 y reportan en `total_income` las cantidades de `9593.119` y `32435.602` como ingreso mensual.

In [57]:
# Agrupando registros de acuerdo a las categorías contenidas en la columna "income_type"
print(df.groupby('income_type')['children'].count()/len(df)*100)
print()
df[df['income_type'] == 'unemployed']


income_type
business                       23.623693
civil servant                   6.778165
employee                       51.656214
entrepreneur                    0.009292
paternity / maternity leave     0.004646
retiree                        17.914053
student                         0.004646
unemployed                      0.009292
Name: children, dtype: float64



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_signed,employed_years
3133,1,38.530190,31.0,secondary education,1,married,0,M,unemployed,1,9593.119,buying property for renting out,+,0.105562
14798,0,45.125895,45.0,bachelor's degree,0,civil partnership,1,F,unemployed,0,32435.602,housing renovation,+,0.123633


Al obtener el % de la muestra, nos encontramos con que este es mínimo en comparación con la muestra completa, apenas representando el `0.009%`. Debido al hecho anterior y a la lógica llevada al reportar ingreso mensual, podemos concluir que pudierón haber sido errores humanos a la hora de recabar la información y en vez de seleccionar la opcion de `employee`, seleccionarón `unemployed`, por ende, procederemos a agregar estos registros a la categoría de `employee` que ademas es la más representativa de la muestra.

In [58]:
# Aborda los valores problemáticos, si existen
df.loc[(df['income_type'] == 'unemployed'),['income_type']] = 'employee'

In [59]:
# Comprueba el resultado - asegúrate de que esté arreglado
df.groupby('income_type')['children'].count()/len(df)*100


income_type
business                       23.623693
civil servant                   6.778165
employee                       51.665505
entrepreneur                    0.009292
paternity / maternity leave     0.004646
retiree                        17.914053
student                         0.004646
Name: children, dtype: float64

El análisis anterior nos conlleva a una conclusión erronea (aparentemente) anteriormente realizada, en la que se suponia que aquellos registros que tienen faltantes los valores correspondientes a `days_employed` y `total_income` se debía a cuestiones de `desempleo`.

Esto nos permite plantear una nueva hipotesis en la que posiblemente al momento de recabar estos datos, en específico estos eran opcionales y los clientes con la intención de mantener la confidencialidad, no han proporcionado dichos datos o por otro lado, son personas que no usan cuentas bancarias de nómina.

In [60]:
# Comprobar los duplicados

print(df.columns)
df['duplicated_row'] = df.duplicated(subset=['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'])
df[df['duplicated_row'] == True]

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose', 'days_employed_signed', 'employed_years'],
      dtype='object')


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_signed,employed_years,duplicated_row
2849,0,NaN,41.0,secondary education,1,married,0,F,employee,0,NaN,purchase of the house for my family,-,NaN,True
3290,0,NaN,58.0,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding,-,NaN,True
4182,1,NaN,34.0,bachelor's degree,0,civil partnership,1,F,employee,0,NaN,wedding ceremony,-,NaN,True
4851,0,NaN,60.0,secondary education,1,civil partnership,1,F,retiree,0,NaN,wedding ceremony,-,NaN,True
5557,0,NaN,58.0,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding,-,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,NaN,64.0,secondary education,1,married,0,F,retiree,0,NaN,supplementary education,-,NaN,True
21032,0,NaN,60.0,secondary education,1,married,0,F,retiree,0,NaN,to become educated,-,NaN,True
21132,0,NaN,47.0,secondary education,1,married,0,F,employee,0,NaN,housing renovation,-,NaN,True
21281,1,NaN,30.0,bachelor's degree,0,married,0,F,employee,0,NaN,buy commercial real estate,-,NaN,True


In [62]:
# Aborda los duplicados, si existen
df[df['duplicated_row']].index

Int64Index([ 2849,  3290,  4182,  4851,  5557,  6312,  7808,  7921,  7938,
             8583,  9238,  9528,  9604,  9627,  9855, 10462, 10697, 10864,
            10994, 11791, 12373, 12375, 12736, 13025, 13639, 13773, 13878,
            13942, 14097, 14432, 14728, 14832, 15091, 15188, 15273, 15991,
            16176, 16204, 16378, 16902, 16904, 17338, 17379, 17755, 17774,
            18328, 18349, 18428, 18521, 18563, 18755, 19041, 19184, 19295,
            19321, 19369, 19387, 19559, 19688, 19832, 19946, 20116, 20165,
            20187, 20297, 20662, 20702, 21032, 21132, 21281, 21415],
           dtype='int64')

In [63]:
df = df.drop(list(df[df['duplicated_row'] == True].index)).reset_index(drop=True)
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_signed,employed_years,duplicated_row
0,1,-8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,-,23.116912,False
1,1,-4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,-,11.026860,False
2,0,-5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,-,15.406637,False
3,3,-4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,-,11.300677,False
4,0,38.843159,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,+,0.106420,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-4529.316663,43.0,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,-,12.409087,False
21450,0,39.262261,67.0,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,+,0.107568,False
21451,1,-2113.346888,38.0,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,-,5.789991,False
21452,3,-3112.481705,38.0,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,-,8.527347,False


In [64]:
# Última comprobación para ver si tenemos duplicados
df[df['duplicated_row'] == True]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   children              21454 non-null  object 
 1   days_employed         19351 non-null  float64
 2   dob_years             21454 non-null  float64
 3   education             21454 non-null  object 
 4   education_id          21454 non-null  int64  
 5   family_status         21454 non-null  object 
 6   family_status_id      21454 non-null  int64  
 7   gender                21454 non-null  object 
 8   income_type           21454 non-null  object 
 9   debt                  21454 non-null  int64  
 10  total_income          19351 non-null  float64
 11  purpose               21454 non-null  object 
 12  days_employed_signed  21454 non-null  object 
 13  employed_years        19351 non-null  float64
 14  duplicated_row        21454 non-null  bool   
dtypes: bool(1), float64

In [65]:
# Comprueba el tamaño del conjunto de datos que tienes ahora, después de haber ejecutado estas primeras manipulaciones
df.shape

(21454, 15)



Este nuevo conjunto contiene registros que no se encuentran duplicados considerando todas su características, incluso las agregadas a traves de la realización de este análisis.

Lo implementado fue que a través del método `df.duplicated()` aplicado a las columnas del `df` original, generamos una nueva columna de referencia llamada `duplicataed_row` con valores bool, una vez teniendo ese dato de referencia y debido a que este método unicamente nos devuelve `True` o `False`, se filtro la tabla que cumpliera esta condición `df['duplicated_row'] == True` con la intención de mirar más a detalle los registros y extraer el `index` de cada uno de ellos. 

Por último, usando el método `drop` y pasando la lista de `index` encontrada anteriormente, eliminamos los registros duplicados para reasignar este último resultado nuevamente a la variable`df`.

# Trabajar con valores ausentes

In [66]:
# Encuentra los diccionarios
print(df.groupby(['family_status','family_status_id'])['children'].count())
print()
print(df.groupby(['education','education_id'])['children'].count())

family_status      family_status_id
civil partnership  1                    4151
divorced           3                    1195
married            0                   12339
unmarried          4                    2810
widow / widower    2                     959
Name: children, dtype: int64

education            education_id
bachelor's degree    0                5250
graduate degree      4                   6
primary education    3                 282
secondary education  1               15172
some college         2                 744
Name: children, dtype: int64


Haciendo enfasis a 4 columnas en particular de las contenidas en el `df` nos encontramos con `education`, `education_id`, `family_status` y `family_status_id`; estas columnas de alguna manera se consideran duplicadas porque ambas representan lo mismo, por ende, es favorable trabajar unicamente con una de ellas con la intención de minimizar información redundante, tiempo de procesamiento y simplificación del `df`.

La manera más adecuada de trabajar con estas columnas es generando diccionarios que nos permitan reconocer a traves de una llave el valor asignado, con propositos de buenas practicas la asignación sera de la siguiente forma:

- Llave: Clave numérica correspondiente a `education_id` y `family_status_id`.
- Valor: Valor correspondiente a `education` y `family_status`

Dando como resultado los siguientes diccionarios:

- `family_status = {0: 'married', 1: 'civil partnership', 2: 'widow / widower', 3: 'divorced', 4: 'unmarried'}`
- `education = {0: 'bachelor's degree', 1: 'secondary education', 2: 'some college', 3: 'primary education', 4: 'graduate degree'}`

### Restaurar valores ausentes en `total_income`



Hasta este momento tenemos 2 columnas que cuentan con valores usentes que son `days_employed` y `total_income`, para el caso de esta última, podemos empezar a hacer un análisis que nos permita generar rangos de edad haciendo uso de `dob_years` y extraer información de ahí para poder identificar que valores serían los más convenientes para resutarar en cada uno de dichos registros.

Como anteriormente se mencionó, el rango de edad considerado en el `df` es de 0 a 75 años, por ende podriamos considerar rangos de 10 años hasta llegar a 70 empezando con una categoría que vaya de 0 a 20 y considerar una categoría llamada `70 y más`.


In [67]:
df = df.astype({'dob_years':'int64'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   children              21454 non-null  object 
 1   days_employed         19351 non-null  float64
 2   dob_years             21454 non-null  int64  
 3   education             21454 non-null  object 
 4   education_id          21454 non-null  int64  
 5   family_status         21454 non-null  object 
 6   family_status_id      21454 non-null  int64  
 7   gender                21454 non-null  object 
 8   income_type           21454 non-null  object 
 9   debt                  21454 non-null  int64  
 10  total_income          19351 non-null  float64
 11  purpose               21454 non-null  object 
 12  days_employed_signed  21454 non-null  object 
 13  employed_years        19351 non-null  float64
 14  duplicated_row        21454 non-null  bool   
dtypes: bool(1), float64

In [68]:
# Vamos a escribir una función que calcule la categoría de edad

def age_range(data):
    if data <= 30:
        return '19 a 30'
    elif data > 30 and data <= 40:
        return '31 a 40'
    elif data > 40 and data <= 50:
        return '41 a 50'
    elif data > 50 and data <= 60:
        return '51 a 60'
    elif data > 60 and data <= 70:
        return '61 a 70'
    else:
        return '70 y más'

In [69]:
# Prueba si la función funciona bien
for i in range(19,76):
    print(age_range(i))

19 a 30
19 a 30
19 a 30
19 a 30
19 a 30
19 a 30
19 a 30
19 a 30
19 a 30
19 a 30
19 a 30
19 a 30
31 a 40
31 a 40
31 a 40
31 a 40
31 a 40
31 a 40
31 a 40
31 a 40
31 a 40
31 a 40
41 a 50
41 a 50
41 a 50
41 a 50
41 a 50
41 a 50
41 a 50
41 a 50
41 a 50
41 a 50
51 a 60
51 a 60
51 a 60
51 a 60
51 a 60
51 a 60
51 a 60
51 a 60
51 a 60
51 a 60
61 a 70
61 a 70
61 a 70
61 a 70
61 a 70
61 a 70
61 a 70
61 a 70
61 a 70
61 a 70
70 y más
70 y más
70 y más
70 y más
70 y más


In [70]:
# Crear una nueva columna basada en la función
df['age_range'] = df['dob_years'].apply(age_range)

In [71]:
# Comprobar cómo los valores en la nueva columna
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_signed,employed_years,duplicated_row,age_range
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,-,23.116912,False,41 a 50
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,-,11.026860,False,31 a 40
2,0,-5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,-,15.406637,False,31 a 40
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,-,11.300677,False,31 a 40
4,0,38.843159,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,+,0.106420,False,51 a 60


In [72]:
# Crea una tabla sin valores ausentes y muestra algunas de sus filas para asegurarte de que se ve bien
df_filtered = df[df['total_income'].isna() != True]
df_filtered.isna().sum()

children                0
days_employed           0
dob_years               0
education               0
education_id            0
family_status           0
family_status_id        0
gender                  0
income_type             0
debt                    0
total_income            0
purpose                 0
days_employed_signed    0
employed_years          0
duplicated_row          0
age_range               0
dtype: int64

In [73]:
# Examina los valores medios de los ingresos en función de los factores que identificaste
df_filtered.pivot_table(values='total_income',index='age_range',columns='children',aggfunc='mean')
df_filtered.pivot_table(values='total_income',index='age_range',columns=['education','education_id'],aggfunc='mean')

education,bachelor's degree,graduate degree,primary education,secondary education,some college
education_id,0,4,3,1,2
age_range,,,,,
19 a 30,30012.887714,NaN,26234.023138,23507.614291,25728.623734
31 a 40,34073.887737,18187.3015,21600.113929,25878.977463,32078.032073
41 a 50,35906.366666,31771.3210,24119.783553,25969.087367,33824.340806
51 a 60,33172.749359,42945.7940,18596.840679,23769.009113,27770.952979
61 a 70,30418.736277,28334.2150,18318.122259,21662.009158,29625.153682
70 y más,26165.386143,NaN,16357.620800,18594.156654,19946.795000


In [74]:
# Examina los valores medianos de los ingresos en función de los factores que identificaste
df_filtered.pivot_table(values='total_income',index='age_range',columns='children',aggfunc='median')
df_filtered.pivot_table(values='total_income',index='age_range',columns=['education','education_id'],aggfunc='median')

education,bachelor's degree,graduate degree,primary education,secondary education,some college
education_id,0,4,3,1,2
age_range,,,,,
19 a 30,26176.5470,NaN,23388.8070,21334.8370,22798.0720
31 a 40,28955.2600,18187.3015,19674.2825,23086.2420,28829.7110
41 a 50,30451.0100,31771.3210,21807.6680,22776.6240,29497.7090
51 a 60,27665.2370,42945.7940,18022.0315,21028.0100,22718.9595
61 a 70,25194.1730,28334.2150,16240.8440,18794.6800,28058.6760
70 y más,26223.0685,NaN,15013.5050,18146.7015,19946.7950




Utilizando `pivot_table` para poder cojuntar `children` y `age_range` aplicando media y mediana a `total_income` que aunque las diferencias son sutiles, si son notables pues podemos observar que aunque los promedios son más altos, la frecuencia y valores medios de `total_income` se encuentran por debajo o igual a los valores mostrados en la tabla anterior en la mayoría de los casos. 

Por otro lado, aplicando de igual forma la tabla pivote `education` en vez de `children`, se observan los mismos resultados, la mediana se encuentra por debajo del promedio en la mayoria de los casos.

Considero que para restaurar los valores de `total_income` sería mejor aplicarlo bajo la operación `median` considerando como clasificador `age_range` como`education` puesto que los ingresos estan más basados en estos parámetros (podría considerarse mejor atributo la columna `days_employed` pero esta presenta más anomalías en sus datos).


In [75]:
#  Escribe una función que usaremos para completar los valores ausentes

def complete_total_income(age_range,education_id):
    global df_filtered
    try:
        return df_filtered[(df_filtered['age_range'] == age_range) & (df_filtered['education_id'] == education_id)]['total_income'].median()
    except:
        return df_filtered[df_filtered['age_range'] == age_range]['total_income'].median()



In [76]:
# Aplícalo a cada fila
df.reset_index(drop=True)

result = []
indexes = df[df['total_income'].isna() == True].index
for i in range(len(df)):
    if i in indexes:
        df.loc[[i],['total_income']] = complete_total_income(df['age_range'][i],df['education_id'][i])
    else:
        df.loc[[i],['total_income']] = df['total_income'][i]


In [77]:
# Comprueba si tenemos algún error
df['total_income'].isna().sum()

0



Se encontrarón diversos errores por la forma de trabajo adoptada, por ende, se considero reiniciar el index del `df` debido a que en pasos anteriores, se eliminarón los datos duplicados; por otro lado, por alguna razón no se pudo trabajar del todo con valores `nan` puesto que todos son el mismo tipo de dato, por ende se decidio crear una lista que contuviera todos los index de aquellos requistros que en la columna `total_income` tuvieran este valor, por último se hizo un recorrido de el `df` haciendo comprobaciones en las que si se encuentra el index dentro de la lista de los registros con valores nulos, se aplica la función `complete_total_income`, caso contrario, se mantiene el dato original.

Esto resulto en un total de 19358 que al sumarle 2096 (cantida de registros que tenían valores nulos) nos da un total de 21454 que es la cantidad de registros en nuestro `df`.


In [78]:
# Reemplazar los valores ausentes si hay algún error
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   children              21454 non-null  object 
 1   days_employed         19351 non-null  float64
 2   dob_years             21454 non-null  int64  
 3   education             21454 non-null  object 
 4   education_id          21454 non-null  int64  
 5   family_status         21454 non-null  object 
 6   family_status_id      21454 non-null  int64  
 7   gender                21454 non-null  object 
 8   income_type           21454 non-null  object 
 9   debt                  21454 non-null  int64  
 10  total_income          21454 non-null  float64
 11  purpose               21454 non-null  object 
 12  days_employed_signed  21454 non-null  object 
 13  employed_years        19351 non-null  float64
 14  duplicated_row        21454 non-null  bool   
 15  age_range          

In [79]:
# Comprobar el número de entradas en las columnas

df.shape

(21454, 16)

###  Restaurar valores en `days_employed`

In [80]:
# Convirtiendo los valores de la columna "days_employed" en positivos
df['days_employed'] = df['days_employed'].abs()
df.head(30)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_signed,employed_years,duplicated_row,age_range
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,-,23.116912,False,41 a 50
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,-,11.026860,False,31 a 40
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,-,15.406637,False,31 a 40
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,-,11.300677,False,31 a 40
4,0,38.843159,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,+,0.106420,False,51 a 60
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,-,2.537495,False,19 a 30
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,-,7.888225,False,41 a 50
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,-,0.418574,False,41 a 50
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,-,18.985932,False,31 a 40
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,-,5.996593,False,41 a 50


In [81]:
df_filtered = df[df['days_employed'].isna() != True]
df_filtered.isna().sum()

children                0
days_employed           0
dob_years               0
education               0
education_id            0
family_status           0
family_status_id        0
gender                  0
income_type             0
debt                    0
total_income            0
purpose                 0
days_employed_signed    0
employed_years          0
duplicated_row          0
age_range               0
dtype: int64

In [82]:
# Distribución de las medianas de `days_employed` en función de los parámetros identificados

df_filtered.pivot_table(values='days_employed',index='age_range',columns='children',aggfunc='median')
df_filtered.pivot_table(values='days_employed',index='age_range',columns=['education','education_id'],aggfunc='median')
df_filtered.pivot_table(values='days_employed',index='age_range',columns=['income_type'],aggfunc='median')


income_type,business,civil servant,employee,entrepreneur,paternity / maternity leave,retiree,student
age_range,,,,,,,
19 a 30,954.067196,1470.574526,1045.363024,520.848083,NaN,41.592260,578.751554
31 a 40,1525.146930,2722.984427,1566.948016,NaN,3296.759962,41.746823,NaN
41 a 50,1930.492791,3452.000013,1953.943699,NaN,NaN,41.846186,NaN
51 a 60,2051.793434,4027.073137,2235.163052,NaN,NaN,41.606898,NaN
61 a 70,2358.275817,3318.440092,2669.073965,NaN,NaN,41.748668,NaN
70 y más,3095.344969,1204.922098,4093.896003,NaN,NaN,41.995093,NaN


In [83]:
# Distribución de las medias de `days_employed` en función de los parámetros identificados

df_filtered.pivot_table(values='days_employed',index='age_range',columns='children',aggfunc='mean')
df_filtered.pivot_table(values='days_employed',index='age_range',columns=['education','education_id'],aggfunc='mean')
df_filtered.pivot_table(values='days_employed',index='age_range',columns=['income_type'],aggfunc='mean')


income_type,business,civil servant,employee,entrepreneur,paternity / maternity leave,retiree,student
age_range,,,,,,,
19 a 30,1207.954723,1696.874626,1263.663549,520.848083,NaN,41.176595,578.751554
31 a 40,1899.538092,2974.058743,2067.362248,NaN,3296.759962,41.622304,NaN
41 a 50,2502.924145,4049.392555,2767.938368,NaN,NaN,41.728410,NaN
51 a 60,2899.605957,4944.394327,3271.309661,NaN,NaN,41.623349,NaN
61 a 70,3644.075107,4390.716537,3883.335727,NaN,NaN,41.699856,NaN
70 y más,4622.013775,1204.922098,4266.205758,NaN,NaN,41.896773,NaN




De acuerdo con la información obtenida y aplicando distintas tablas pivote, considero que los parámetro estrechamente relacionado con la columna `days_employed` son `age_range` y `type_income`, esto debido a la edad que tienen los clientes se puede determinar que tan errado o cierta es la cantidad de días que lleva empleado.

Para este proceso `median` nos proporciona los datos más realistas de acuerdo al análisis hasta este momento, `mean` nos aleja por completo de `median` esto nos da a pensar que hay valores extremadamente altos en las muestras que provocan sesgos.

In [84]:
# Escribamos una función que calcule medias o medianas (dependiendo de tu decisión) según el parámetro identificado

def complete_days_employed(age_range,income_type):
    global df_filtered
    try:
        return df_filtered[(df_filtered['age_range'] == age_range) & (df_filtered['income_type'] == income_type)]['days_employed'].median()
    except:
        return df_filtered[df_filtered['age_range'] == age_range]['days_employed'].median()


In [85]:
print(df['income_type'].unique())

['employee' 'retiree' 'business' 'civil servant' 'entrepreneur' 'student'
 'paternity / maternity leave']


In [86]:
# Comprueba que la función funciona
print(complete_days_employed('19 a 30','student'))


578.7515535382181


In [87]:
# Aplicar la función al income_type
print(complete_days_employed('19 a 30','paternity / maternity leave'))


nan


In [88]:
# Comprueba si la función funcionó

result = []
indexes = df[df['days_employed'].isna() == True].index
for i in range(len(df)):
    if i in indexes:
        result.append(complete_days_employed(df['age_range'][i],df['income_type'][i]))
    else:
        result.append(df['days_employed'][i])

print(result)

[8437.673027760233, 4024.803753850451, 5623.422610230956, 4124.747206540018, 38.84315890945456, 926.1858308789184, 2879.202052139952, 152.7795691752705, 6929.865298973741, 2188.7564450779378, 4171.483646903305, 792.7018870609315, 41.74866776882945, 1846.6419410560736, 1844.9561821875543, 972.3644189900732, 1719.9342261530542, 2369.999720209008, 45.694193711479315, 10038.818548915877, 1311.6041664289485, 253.68516578402495, 1766.6441376815046, 272.9813850334044, 38.647483209060404, 41.50096910369815, 3452.000013033826, 529.1916346594236, 717.2743239418727, 41.74866776882945, 38.30840964782823, 1682.08343808475, 4649.910831728614, 1548.63754364046, 4488.067031277062, 44.97957445020493, 176.21668833712155, 6448.810859676204, 597.8818273051091, 650.5877964026586, 1030.3626056014652, 3452.000013033826, 1257.4961901986044, 4375.681384360526, 1362.0417275002387, 1039.4517405766287, 2262.712303593806, 2689.1372742032745, 3341.0678863600638, 1181.443228272809, 40.38030049523952, 6953.6316545631

In [89]:
# Reemplazar valores ausentes

indexes = df[df['days_employed'].isna() == True].index
for i in range(len(df)):
    if i in indexes:
        df.loc[[i],['days_employed']] = complete_days_employed(df['age_range'][i],df['income_type'][i])
    else:
        df.loc[[i],['days_employed']] = df['days_employed'][i]

In [90]:
# Comprueba las entradas en todas las columnas: asegúrate de que hayamos corregido todos los valores ausentes
df[df['days_employed'].isna() == True]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_signed,employed_years,duplicated_row,age_range
5931,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,27665.237,buy residential real estate,-,NaN,False,51 a 60


In [91]:
df.loc[[5931],['days_employed']] = df[df['age_range'] == '51 a 60']['days_employed'].median()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   children              21454 non-null  object 
 1   days_employed         21454 non-null  float64
 2   dob_years             21454 non-null  int64  
 3   education             21454 non-null  object 
 4   education_id          21454 non-null  int64  
 5   family_status         21454 non-null  object 
 6   family_status_id      21454 non-null  int64  
 7   gender                21454 non-null  object 
 8   income_type           21454 non-null  object 
 9   debt                  21454 non-null  int64  
 10  total_income          21454 non-null  float64
 11  purpose               21454 non-null  object 
 12  days_employed_signed  21454 non-null  object 
 13  employed_years        19351 non-null  float64
 14  duplicated_row        21454 non-null  bool   
 15  age_range          

In [92]:
print(len(df['days_employed'].unique()))

19361


## Clasificación de datos


In [93]:
# Muestra los valores de los datos seleccionados para la clasificación
df.info()
#df.loc[:,['children','education','education_id','family_status','family_status_id','gender','purpose','age_range']]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   children              21454 non-null  object 
 1   days_employed         21454 non-null  float64
 2   dob_years             21454 non-null  int64  
 3   education             21454 non-null  object 
 4   education_id          21454 non-null  int64  
 5   family_status         21454 non-null  object 
 6   family_status_id      21454 non-null  int64  
 7   gender                21454 non-null  object 
 8   income_type           21454 non-null  object 
 9   debt                  21454 non-null  int64  
 10  total_income          21454 non-null  float64
 11  purpose               21454 non-null  object 
 12  days_employed_signed  21454 non-null  object 
 13  employed_years        19351 non-null  float64
 14  duplicated_row        21454 non-null  bool   
 15  age_range          

In [94]:
# Comprobar los valores únicos
print(df.nunique())

children                    6
days_employed           19361
dob_years                  57
education                   5
education_id                5
family_status               5
family_status_id            5
gender                      2
income_type                 7
debt                        2
total_income            19356
purpose                    38
days_employed_signed        2
employed_years          19351
duplicated_row              1
age_range                   6
dtype: int64




De acuerdo a la información que arroja la sentencia `nunique()` podemos concluir que las siguientes columnas son categóricas:
- `children`
- `education`
- `education_id`
- `family_status`
- `family_status_id`
- `gender`
- `income_type`
- `debt`
- `purpose`
- `age_range`

In [95]:
# Escribamos una función para clasificar los datos en función de temas comunes
# Los ingresos van desde 3306.762 hasta 362496.645
df['total_income'].sort_values()
def range_income(data):
    if data <= 10000:
        return 'de 3 mil a 10 mil'
    elif data > 10000 and data <= 20000:
        return 'de 10 mil a 20 mil'
    elif data > 20000 and data <= 30000:
        return 'de 20 mil a 30 mil'
    elif data > 30000 and data <= 40000:
        return 'de 30 mil a 40 mil'
    elif data > 40000 and data <= 50000:
        return 'de 40 mil a 50 mil'
    elif data > 50000 and data <= 60000:
        return 'de 50 mil a 60 mil'
    elif data > 60000 and data <= 70000:
        return 'de 60 mil a 70 mil'
    elif data > 70000 and data <= 80000:
        return 'de 70 mil a 80 mil'
    elif data > 90000 and data <= 100000:
        return 'de 90 mil a 100 mil'
    elif data > 100000 and data <= 110000:
        return 'de 100 mil a 110 mil'
    elif data > 110000 and data <= 120000:
        return 'de 110 mil a 120 mil'
    elif data > 120000 and data <= 130000:
        return 'de 120 mil a 130 mil'
    elif data > 130000 and data <= 140000:
        return 'de 130 mil a 140 mil'
    elif data > 140000 and data <= 150000:
        return 'de 140 mil a 150 mil'
    elif data > 150000 and data <= 160000:
        return 'de 150 mil a 160 mil'
    elif data > 160000 and data <= 170000:
        return 'de 160 mil a 170 mil'
    elif data > 180000 and data <= 190000:
        return 'de 180 mil a 190 mil'
    elif data > 190000 and data <= 200000:
        return 'de 190 mil a 200 mil'
    elif data > 200000 and data <= 210000:
        return 'de 200 mil a 210 mil'
    elif data > 210000 and data <= 220000:
        return 'de 210 mil a 220 mil'
    elif data > 220000 and data <= 230000:
        return 'de 220 mil a 230 mil'
    elif data > 230000 and data <= 240000:
        return 'de 230 mil a 240 mil'
    elif data > 240000 and data <= 250000:
        return 'de 240 mil a 250 mil'
    elif data > 250000 and data <= 260000:
        return 'de 250 mil a 260 mil'
    elif data > 260000 and data <= 270000:
        return 'de 260 mil a 270 mil'
    elif data > 270000 and data <= 280000:
        return 'de 270 mil a 280 mil'
    elif data > 280000 and data <= 290000:
        return 'de 280 mil a 290 mil'
    elif data > 290000 and data <= 300000:
        return 'de 290 mil a 300 mil'
    elif data > 300000 and data <= 310000:
        return 'de 300 mil a 310 mil'
    elif data > 310000 and data <= 320000:
        return 'de 310 mil a 320 mil'
    elif data > 320000 and data <= 330000:
        return 'de 320 mil a 330 mil'
    elif data > 330000 and data <= 340000:
        return 'de 330 mil a 340 mil'
    elif data > 340000 and data <= 350000:
        return 'de 340 mil a 350 mil'
    elif data > 350000 and data <= 360000:
        return 'de 350 mil a 360 mil'
    else:
        return 'de 300 mil a 310 mil'

In [96]:
# Crea una columna con las categorías y cuenta los valores en ellas
df['range_income'] = df['total_income'].apply(range_income)


In [97]:
# Revisar todos los datos numéricos en la columna seleccionada para la clasificación
df.head(15)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_signed,employed_years,duplicated_row,age_range,range_income
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,-,23.116912,False,41 a 50,de 40 mil a 50 mil
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,-,11.026860,False,31 a 40,de 10 mil a 20 mil
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,-,15.406637,False,31 a 40,de 20 mil a 30 mil
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,-,11.300677,False,31 a 40,de 40 mil a 50 mil
4,0,38.843159,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,+,0.106420,False,51 a 60,de 20 mil a 30 mil
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,-,2.537495,False,19 a 30,de 40 mil a 50 mil
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,-,7.888225,False,41 a 50,de 30 mil a 40 mil
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,-,0.418574,False,41 a 50,de 20 mil a 30 mil
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,-,18.985932,False,31 a 40,de 10 mil a 20 mil
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,-,5.996593,False,41 a 50,de 20 mil a 30 mil


In [98]:
# Obtener estadísticas resumidas para la columna
df.groupby('range_income')['range_income'].count().sort_values(ascending=False)


range_income
de 20 mil a 30 mil      7856
de 10 mil a 20 mil      6628
de 30 mil a 40 mil      3232
de 40 mil a 50 mil      1492
de 3 mil a 10 mil        926
de 50 mil a 60 mil       648
de 60 mil a 70 mil       294
de 70 mil a 80 mil       156
de 300 mil a 310 mil      89
de 90 mil a 100 mil       40
de 100 mil a 110 mil      34
de 110 mil a 120 mil      16
de 130 mil a 140 mil      12
de 120 mil a 130 mil       6
de 160 mil a 170 mil       5
de 270 mil a 280 mil       3
de 150 mil a 160 mil       3
de 140 mil a 150 mil       3
de 190 mil a 200 mil       2
de 200 mil a 210 mil       2
de 180 mil a 190 mil       2
de 210 mil a 220 mil       1
de 240 mil a 250 mil       1
de 250 mil a 260 mil       1
de 350 mil a 360 mil       1
de 220 mil a 230 mil       1
Name: range_income, dtype: int64

In [99]:
# Crear una función para clasificar en diferentes grupos `purpose`

purpose = {
    'Wedding': ['wedding ceremony','having a wedding','to have a wedding',''],
    'Real State': ['real estate transactions','transactions with my real estate','purchase of the house for my family',
                  'buy real estate','property','buy residential real estate','purchase of my own house','purchase of the house','housing transactions'],
    'Build Real State': ['building a property','construction of own property','building a real estate'],
    'Comercial Real State': ['buying property for renting out','transactions with commercial real estate','buy commercial real estate',
                            ],
    'Education': ['to get a supplementary education','getting higher education','profile education','going to university',
                 'university education','to become educated','supplementary education','getting an education','education'],
    'Car': ['second-hand car purchase','purchase of a car','buying my own car','to buy a car','cars','car purchase','car',
           'buying a second-hand car','to own a car'],
    'Remodelation': ['housing','housing renovation']
}

def purpose_classification(data):
    for i in purpose:
        if data in purpose[i]:
            return i


In [100]:
# Crear una columna con categorías
df['purpose_categories'] = df['purpose'].apply(purpose_classification)
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_signed,employed_years,duplicated_row,age_range,range_income,purpose_categories
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,-,23.116912,False,41 a 50,de 40 mil a 50 mil,Real State
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,-,11.026860,False,31 a 40,de 10 mil a 20 mil,Car
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,-,15.406637,False,31 a 40,de 20 mil a 30 mil,Real State
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,-,11.300677,False,31 a 40,de 40 mil a 50 mil,Education
4,0,38.843159,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,+,0.106420,False,51 a 60,de 20 mil a 30 mil,Wedding
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,-,2.537495,False,19 a 30,de 40 mil a 50 mil,Real State
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,-,7.888225,False,41 a 50,de 30 mil a 40 mil,Real State
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,-,0.418574,False,41 a 50,de 20 mil a 30 mil,Education
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,-,18.985932,False,31 a 40,de 10 mil a 20 mil,Wedding
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,-,5.996593,False,41 a 50,de 20 mil a 30 mil,Real State


In [101]:
# Contar los valores de cada categoría para ver la distribución
print(df[df['purpose_categories'].isna() == True]['purpose'].unique())
df.info()


[]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   children              21454 non-null  object 
 1   days_employed         21454 non-null  float64
 2   dob_years             21454 non-null  int64  
 3   education             21454 non-null  object 
 4   education_id          21454 non-null  int64  
 5   family_status         21454 non-null  object 
 6   family_status_id      21454 non-null  int64  
 7   gender                21454 non-null  object 
 8   income_type           21454 non-null  object 
 9   debt                  21454 non-null  int64  
 10  total_income          21454 non-null  float64
 11  purpose               21454 non-null  object 
 12  days_employed_signed  21454 non-null  object 
 13  employed_years        19351 non-null  float64
 14  duplicated_row        21454 non-null  bool   
 15  age_range       

## Comprobación de las hipótesis


**¿Existe una correlación entre tener hijos y pagar a tiempo?**

In [102]:
# Comprueba los datos sobre los hijos y los pagos puntuales
# Calcular la tasa de incumplimiento en función del número de hijos

# Convirtiendo a tipo de dato "int64" para poder ordenar
df.loc[df['children'] == "5 o más",'children'] = 5

cumple = df[df['debt'] == 0].groupby('children')['children'].count()
no_cumple = df[df['debt'] == 1].groupby('children')['children'].count()

pd.DataFrame({'cumple': cumple,
              'no cumple': no_cumple,
              'porcentaje': no_cumple/(no_cumple+cumple)*100})

,cumple,no cumple,porcentaje
children,,,
0,13074,1064,7.525817
1,4364,444,9.234609
2,1858,194,9.454191
3,303,27,8.181818
4,37,4,9.756098
5,77,8,9.411765


**Conclusión**

No podemos apoyar el hecho de que hay una relación entre el número de hijos y el cumplimiento de pago a tiempo de acuerdo a nuestros datos, si bien, se puede observar que las personas que tienen `0` hijos tienden a cumplir un poco más en tiempo y forma con sus pagos, esto no permite asumir que a mayor hijos menor cumplimiento ni viceversa.

**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [103]:
# Comprueba los datos del estado familiar y los pagos a tiempo
# Calcular la tasa de incumplimiento basada en el estado familiar

cumple = df[df['debt'] == 0].groupby('family_status')['family_status'].count()
no_cumple = df[df['debt'] == 1].groupby('family_status')['family_status'].count()

pd.DataFrame({'cumple': cumple,
              'no cumple': no_cumple,
              'porcentaje': no_cumple/(no_cumple+cumple)*100}).sort_values('porcentaje')


,cumple,no cumple,porcentaje
family_status,,,
widow / widower,896,63,6.569343
divorced,1110,85,7.112971
married,11408,931,7.545182
civil partnership,3763,388,9.347145
unmarried,2536,274,9.750890


**Conclusión**

Por otro lado, aparentemente, las personas que han tenido alguna estado familiar de compromiso, a diferencia de las parejas homosexuales, tienden a tener un mayor cumplimiento con el pago a tiempo en comparación de los que se encuentran solteros.

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [104]:
# Comprueba los datos del nivel de ingresos y los pagos a tiempo
# Calcular la tasa de incumplimiento basada en el nivel de ingresos

cumple = df[df['debt'] == 0].groupby('range_income')['range_income'].count()
no_cumple = df[df['debt'] == 1].groupby('range_income')['range_income'].count()

pd.DataFrame({'cumple': cumple,
              'no cumple': no_cumple,
              'porcentaje': no_cumple/(no_cumple+cumple)*100}).sort_values('porcentaje')


,cumple,no cumple,porcentaje
range_income,,,
de 100 mil a 110 mil,33.0,1.0,2.941176
de 90 mil a 100 mil,38.0,2.0,5.000000
de 70 mil a 80 mil,148.0,8.0,5.128205
de 60 mil a 70 mil,278.0,16.0,5.442177
de 110 mil a 120 mil,15.0,1.0,6.250000
de 3 mil a 10 mil,868.0,58.0,6.263499
de 300 mil a 310 mil,83.0,6.0,6.741573
de 40 mil a 50 mil,1390.0,102.0,6.836461
de 30 mil a 40 mil,2985.0,247.0,7.642327


**Conclusión**

Observando la tabla, parecería que aquellas personas que tienen salarios más altos, tienden a cumplir en tiempo el pago, sin embargo, algo que podemos rescatar de esta distribución es que no tenemos los registros suficientes para dar esto como un hecho, pues en comparación con aquellos que si, la cantidad de registros es mínima.

En conclusión nuevamente no podemos decir que hay una estrecha relación entre el rango de ingreso mensual y el pago a tiempo.

**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [105]:
# Consulta los porcentajes de tasa de incumplimiento para cada propósito del crédito y analízalos

cumple = df[df['debt'] == 0].groupby('purpose_categories')['purpose_categories'].count()
no_cumple = df[df['debt'] == 1].groupby('purpose_categories')['purpose_categories'].count()

pd.DataFrame({'cumple': cumple,
              'no cumple': no_cumple,
              'porcentaje': no_cumple/(no_cumple+cumple)*100}).sort_values('porcentaje')


,cumple,no cumple,porcentaje
purpose_categories,,,
Remodelation,1172,81,6.464485
Real State,5312,406,7.100385
Build Real State,1734,144,7.667732
Comercial Real State,1811,151,7.696228
Wedding,2138,186,8.003442
Education,3643,370,9.220035
Car,3903,403,9.359034


**Conclusión**

Con esta distribución podemos concluir lo siguiente:

- Los prestamos relacionados con un bien inmueble, son los que tienden a tener mayor indice de cumplimiento de pago a tiempo que los que tienen propósitos diversos.
- Los propositos que presentan mayor indice de incumplimiento de pago a tiempo son `car`, `education` y `wedding`, lo cual refuerza la conclusión anterior.
- Aquellos prestamos más solicitados son lo `real state` (compra de un bien inmueble para uso familiar) y `car`, de los cuales el primero presenta mayor tasa de cumplimiento de pago a tiempo que el segundo, esto refuerza las dos conclusiones anteriores.

Algo que hay que resaltar es que aunque son un hecho las conclusiones anteriores, la diferencia de tasa de cumplimiento no varía más allá de los 3 puntos entre la de mayor cumplimiento y la de menor cumplimiento, lo que nos dice que en general los registros que se tienen en esta `dataframe` tienden al cumplimiento del pago a tiempo independientemente del proposito del crédito solicitado.


# Conclusión general 

En conclusión a pesar de ser un set de datos relativamente pequeño pudimos obtener muchisima información relacionada con respecto al tipo de características que un banco puede llegar a tomar en cuenta al momento de evaluar la viabilidad de realizar un prestamo a los clientes. La premisa inicial fue el planteamiento de que dependiendo de la cantidad de hijos que tuviera el cliente, podía tener una tendencia al incumplimiento de pago a tiempo de un crédito; basado en la experiencia personal de entrada llegue a pensar que a medida que el cliente tuviera más hijos tendía a incumplir con el pago de los prestamos puesto que el ingreso que genera mes a mes se ve afectado por dicha responsabilidad.

Por otro lado, nos encontramos con diversos problemas como los valores ausentes, duplicados, categorización y atípicos los cuales debierón ser trabajados de distintas formas.

Para el caso de los datos `duplicados` la manera de abordarlos fue sencilla puesto que coincidierón por completo en el registro, lo cual no nos llevo a buscar un patrón columna por columna para poder intentar corregirlos.

Con respecto a los valores `atípicos` el trabajo fue un poco diferente, se encontrarón 3 casos atípidos, empezando por la columna `children` donde se tenía como valor `-1` y `20`.

- Al tratase del caso de `-1` se llego a pensar que al momento de recabar los datos posiblemente se trataban de personas que pudierón perder un hijo, sin embargo en ningún punto del documento esto fue aclarado, por ende y basado en el análisis, se decidio juntarlo con la categoría de mayor ponderación que es el caso de `0` hijos.
- Para el caso de `20`, este valor se considero completamente fuera de rango, de acuerdo con google, la familia promedio es de cuatro integrantes y como máximo es de 9, por lo tanto, y al no tener características intermedias entre 5 y 20, se juntarón estas dos asumiendo que no tendría mayor impacto en el análisis puesto que la muestra fue mínima (menor al `1%`).

Continuando con los valores `atípicos`, también se encontrarón que muchos de los registros presentados en el set de datos, eran incongruentes al relacionalos con otros y tal fue el caso de `days_employed` y `dob_years`. El primero de ellos nos da la información de la cantidad de días que lleva empleado el cliente, por otro lado el segundo parámetro nos especifíca la edad del cliente, sin embargo nos encontramos con datos en los que se reportaban cantidad de días que sobrepasaban los años reportados del cliente y por mucho, llegando incluso al hacer la conversión (`days_empooyed`/365) clientes con 1100 años trabajados lo cual no es congruente en ningún caso, por lo tanto se procedio a un análisis más profundo de dichos valores llegando a la conclusión que el punto decimal era el problema y por ende, se decidio recorrer este mismo dos decimales a la izquierda resultando en valores completamente congruentes con el resto de registros.

El último caso `atípico` encontrado, nuevamente se relaciona con `days_employed` pues la gran mayoría de estos datos se encontraban registrados con valores negativos, llegando a la conclusión de que fue un problema al momento de realizar la captura de datos puesto que al hacer la conversión anteriormente mencionada, se consiguieron datos congruentes.

Respecto a la `categorización`, esta fue requerida en dos ocasiones, la primera en la columna `education` y la segunda en `purpose`, puesto que estas categorizaciones pudieron ser reducidas debido a que el significado de muchas de estas categorías se referia a lo mismo.

Por último tenemos los valores `ausentes` que fue lo más complicado de trabajar, estos estaban presentes en las columnas `days_employed` y `total_income`. En ambos casos, se decidió trabajar con `mediana` esto por razones en las que el `promedio` disparaba por completo los valores proporcionando sesgos al momento de ser tomados en cuenta.

- Para el caso de `total_income` se decidio hacer una revaloración de los posibles valores para poder rellenar los registros y se concluyo a traves de tablas pivote que aquellos que generaban mayor impacto eran en conjunto `age_range` y `education`.
- Para el caso de `days_employed` se concluyo, nuevamente, a traves de tablas pivote que las columnas que generaban mayor impacto nuevamente era `age_range` en conjunto con `type_income`.

De forma general, el `data frame` que estudiamos a lo largo de este proyecto, presenta en su mayoria registros de cumplimiento de pago a tiempo de los clientes independientemente de su estatus familiar, la cantidad de hijos que pudiera tener e inclusive el ingreso mensual. Por otro lado pareciera encontrarse una relación directa con el proposito del prestamo en la que cualquier propósito relacionado con un prestamo para un bien inmueble, tendía a un menor incumplimiento de pago que aquellos que tenían un proposito más específico como es el caso de un prestamo automotriz en el que tiene una alta solicitud por los clientes y es el que presenta mayor incumplimiento de acuerdo a la información proporcionada.

La propuesta de ejercicio que se llevo en este proyecto en definitiva puso a prueba todo lo aprendido en estos 2 sprints, ademas también, obligo a buscar en documentación oficial e incluso ejemplos de otros análisis con el fin de retroalmentarme y poder abarcar la mayor cantidad de posibilidades en los distintos ejercicios requeridos.